In [4]:
!pip install opencv-python
!pip install imutils

In [5]:
class MotionDetector:
    def __init__(self, DELTA_THRESH = 5, MIN_AREA = 5000):
        self.avg = None
        self.DELTA_THRESH = DELTA_THRESH
        self.MIN_AREA = MIN_AREA

    def detect(self, frame):
        frame = imutils.resize(frame, width=500)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        if self.avg is None:
            self.avg = gray.copy().astype('float')
            return

        cv2.accumulateWeighted(gray, self.avg, 0.5)
        frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(self.avg))

        thresh = cv2.threshold(frameDelta, self.DELTA_THRESH, 255,
                           cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        for cnt in cnts:
            if cv2.contourArea(cnt) < self.MIN_AREA:
                continue

            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow('frame', frame)


In [6]:
import cv2, time, imutils

cap = cv2.VideoCapture(0)

# time.sleep(2)

detector = MotionDetector()
while True:
    ret, frame = cap.read()

    detector.detect(frame)


    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()
exit()